In [8]:
import re
import gensim
import logging
import nltk.data
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec

import warnings
warnings.filterwarnings('ignore')

First of all, preprocessing of some piece from 'Crime and Punishment' by Dostoevsky.

In [1]:
from nltk.stem import WordNetLemmatizer
from nltk import sent_tokenize
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import re

from nltk.tokenize import wordpunct_tokenize
from pymorphy2 import MorphAnalyzer

file=open("/Users/almar/OneDrive/Рабочий стол/python/prestuplenie_i_nakazanie1.txt", mode='r+')
r = file.read()

r_low = r.lower() #lowering
no_punc = re.sub(r'[^\w\s]','',r_low) #no punc

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\almar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
from pymystem3 import Mystem
m = Mystem()

In [6]:
lines = no_punc.splitlines()
pin_lines = []
for l in lines:
    lemmas = m.lemmatize(l)
    new_line = ''.join(lemmas)
    pin_lines.append(new_line) #split by lines and lemmatize
pin2 ='\n'.join(pin_lines)

In [7]:
my_file = open("pin_lem.txt", "w+")
my_file.write(pin2)
my_file.close()

In [3]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [18]:
f = open("/Users/almar/pin_lem.txt", mode = 'r')
data = gensim.models.word2vec.LineSentence(f)

Initially, I started with the following parameteres: size = 100, window = 5, min_count = 2, epochs = 10. It seems that the most suitable combination for the corpus is 300, 5, 2, 35. Size less than 300 results in unwanted output of prepositions and pronouns rather than meaningful words (especially for wv.most_similar). Window size of 5 remained default since increase of window size (e.g. 8-10) leads to unwanted results as well. It appears that the higher number of interation (in this case 35), the higher chance to exclude unwanted words like prepositions. 

In [287]:
%time model_pin = gensim.models.Word2Vec(data, vector_size=300, window=5, min_count=2, epochs = 35)

Wall time: 1.76 s


In [288]:
model_path = "pin.bin"
model_pin.wv.save_word2vec_format(model_path, binary=True)

Let's count the number of words in model_pin:

In [360]:
from gensim.models import KeyedVectors
word_vectors = model_pin.wv

word_vectors.save("word2vec.wordvectors")
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')
vector = wv['раскольников']

print(len(model_pin.wv))

1703


As we remember, Raskolnikov is a poor ex-law student who lives in a tiny garret, wearing old clothes due to lack of money. He is also not very kind to other people. Let's see if Raskolnikov would be close to vectors ['каморка', 'лохмотья'] substract ['добрый'].

In [315]:
model_pin.wv.most_similar(positive=["каморка", "лохмотья"], negative=["добрый"], topn=3)

[('раскольников', 0.9935407042503357),
 ('комната', 0.9933441877365112),
 ('глаз', 0.9911273121833801)]

It is a succsess to find 'раскольников' on top, which is so close to vectors (almost 1). The result 'комната' also seems to be relevant to 'каморка'. We can also check how the model works for separate words like 'бедность'. I would suggest the following result as a relative success as the model defines 'нищета' as the second closest word to the vector.

In [306]:
model_pin.wv.most_similar(positive=["бедность"], topn=3)

[('отсюда', 0.9975753426551819),
 ('нищета', 0.9974064230918884),
 ('гордый', 0.9967379570007324)]

Next, let's see how 'раскольников' and 'топор' semanticaly close to each other. The cosine similarity of ~0,88 indicates that these words are almost similar to each other, which is a success, as we remember from the story.

In [337]:
model_pin.wv.similarity("раскольников", "топор")

0.88679916

The following step shows that among words 'мысль', 'процентщица', 'топор', 'убивать', 'раскольников' the model indicates 'раскольников' as not matching one, which is quite unexpected (my expectation was 'мысль'). It seems like the model will output 'раскольников' every time this word is in the vector.

In [374]:
model_pin.wv.doesnt_match("мысль процентщица топор убивать раскольников".split())

'раскольников'

Then, let's visualize the results.

In [369]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [332]:
words = list(model_pin.wv)
print(words)

KeyError: "Key '1703' not present"